In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('./input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./input/test_rejection.csv
./input/comment_input.csv


In [2]:
import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('comment_nlp').config("spark.driver.memory", "6g").getOrCreate()

21/09/12 13:45:36 WARN Utils: Your hostname, zmediatech-huytq11 resolves to a loopback address: 127.0.0.1; using 192.168.1.17 instead (on interface wlp3s0)
21/09/12 13:45:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/12 13:45:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
rawData = spark.read.csv('./input/comment_input.csv', inferSchema=True, header=True)

rawData.show()

+--------------------+------+
|             content|status|
+--------------------+------+
|          có mẹ nuôi|     1|
|   Happy new year..!|     1|
|31/12/2020 co ai ...|     1|
|     00h00’ 1/1/2021|     1|
|         2021!!!!!!!|     1|
|Có hát ngay đoạn đầu|     1|
|        tang 1 nguoi|     1|
|Happy new year 🎉...|     0|
| tang em ng iu cua a|     1|
|ai vừa xem xong v...|     1|
|      happy new year|     1|
|A DiĐà Phật, happ...|     1|
|Giờ không ai nhớ ...|     1|
|Từ khi mất e đến ...|     1|
|   năm mới rồi mn ơi|     1|
|  phi điểu và ve sầu|     1|
|1/1/2021❤ yêu tất...|     1|
|        Sơn tùng mtp|     1|
|    1-1-2021 tykmmmm|     1|
|          1/1/2021:3|     1|
+--------------------+------+
only showing top 20 rows



In [196]:
rawData.printSchema()

root
 |-- content: string (nullable = true)
 |-- status: integer (nullable = true)



In [197]:
rawData.summary().show()

+-------+-------------------+------------------+
|summary|            content|            status|
+-------+-------------------+------------------+
|  count|            1019031|           1019031|
|   mean|           Infinity|  2.15498645281645|
| stddev|                NaN|0.3618919462764307|
|    min|! ! ! ! !!!!!!!!!!!|                 2|
|    25%|       9.25338341E8|                 2|
|    50%|      2.194668604E9|                 2|
|    75%|             1.0E14|                 2|
|    max|         🫔👍👍👍👍|                 3|
+-------+-------------------+------------------+



In [198]:
# !pip install seaborn

In [199]:
# import seaborn as sns

# sns.distplot(rawData.select('status').toPandas())

In [200]:
# rawData.groupBy('status').count().toPandas()

In [5]:
SPLITTED_PATH = './splitted'

rawData.repartition(20).write.partitionBy("status").csv(SPLITTED_PATH, mode="overwrite")

for dirname, _, filenames in os.walk(SPLITTED_PATH):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./splitted/._SUCCESS.crc
./splitted/_SUCCESS
./splitted/status=1/part-00009-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/.part-00012-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv.crc
./splitted/status=1/part-00006-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/part-00002-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/part-00015-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/part-00011-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/.part-00006-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv.crc
./splitted/status=1/.part-00001-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv.crc
./splitted/status=1/.part-00014-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv.crc
./splitted/status=1/part-00014-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/part-00012-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv
./splitted/status=1/.part-00004-290de1a8-e3f0-4f75-bb2a-93d642a95b23.c000.csv.crc
./spli

In [6]:
schemaDdl = "content STRING, status INTEGER"

sparkReader = spark.read.schema(schemaDdl)

splittedData = sparkReader.csv(SPLITTED_PATH)

In [7]:
from pyspark.sql import functions as f
from pyspark.sql.functions import udf
import re
import html

hashtag_regex = r"#(\w{1,})"
url_regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
not_char_regex = "[^a-zA-ZaAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ]"

@udf
def html_unescape(s: str):
    return html.unescape(s)

processedData = (
    splittedData
    .withColumn("content", f.regexp_replace(f.col("content"), hashtag_regex, "$1"))
    .withColumn("content", f.regexp_replace(f.col("content"), url_regex, ""))
    .withColumn("content", f.regexp_replace(f.col("content"), email_regex, ""))
    .withColumn("content", html_unescape("content"))
    .withColumn("content", f.regexp_replace(f.col("content"), not_char_regex, " "))
    .withColumn("content", f.regexp_replace(f.col("content"), " +", " "))
    .withColumn("content", f.trim(f.col("content")))
).filter("content != ''").select("content", "status").coalesce(3)

processedData.show(100)

+--------------------+------+
|             content|status|
+--------------------+------+
|tìm ny ở đây tui ...|     1|
|tên bai hát xịn x...|     1|
|K hay b có thể k ...|     1|
| lượt xem tụt kìa mn|     1|
|        Chúc mừng em|     1|
|tất nhiên hay cho...|     1|
|người giờ còn đây...|     1|
|Nghe thấm thiệt ắ...|     1|
|nhật bản nghệ an ...|     1|
|      Mai yeu TFBOYS|     1|
|Anh hát bài nào c...|     1|
|Bài Sứ Thanh Hoa ...|     1|
|hình như là nhật bản|     1|
|Mọi người ráng lê...|     1|
|Vì ngày hôm nay a...|     1|
|        Thương a quá|     1|
|       nhớ lắm nyc ạ|     1|
|        ĐỈNH THẬT SỰ|     1|
|        nghe mà buồn|     1|
|cảm ơn em va moi ...|     1|
|  soái ca thái quynh|     1|
|        Hay lam luôn|     1|
|bài anh mơ của an...|     1|
|nghe mà đúng t tr...|     1|
|         dể thuog nè|     1|
|        vẫn còn nghe|     1|
| thanh xuân tươi đẹp|     1|
|             Hay quá|     1|
|Các e giữ gìn sức...|     1|
| bài lần này dịu quá|     1|
|nghe hơn 

In [8]:
(trainingData, validationData, testData) = processedData.randomSplit([0.90, 0.1, 0.1], seed=2021)

In [205]:
# from vncorenlp import VnCoreNLP

# annotator = VnCoreNLP("./VnCoreNLP/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx2g') 

# text = "Bùi Thị Bích Phương, thường được biết đến với tên gọi quen thuộc là Bích Phương, là một nữ ca sĩ và giám khảo chương trình truyền hình người Việt Nam."

# word_segmented_text = annotator.tokenize(text)[0]

# print(word_segmented_text)

In [206]:
# !pip install pyvi

In [11]:
from pyspark.sql.functions import col, lower, udf, lit
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql import DataFrame
from pyvi import ViTokenizer, ViPosTagger

tokenize_text_udf = udf(lambda s: ViPosTagger.postagging(ViTokenizer.tokenize(s))[0], ArrayType(StringType()))
def lower_arr_str(x):
    res = []
    for x_ in x:
        res.append(x_.lower())
    return res
to_lower_str_arr = udf(lower_arr_str, ArrayType(StringType()))

def tokenize(df : DataFrame) -> DataFrame:
    return df.withColumn("content1", to_lower_str_arr(tokenize_text_udf(col("content"))))

trainingData = tokenize(trainingData)

trainingData.show()

+--------------------+------+--------------------+
|             content|status|            content1|
+--------------------+------+--------------------+
|A A chị tôi sao đ...|     1|[a, a, chị, tôi, ...|
|  A Amazing Good job|     1|[a_amazing, good_...|
|A B C D I F J K I...|     1|[a, b, c, d, i, f...|
|       A Bo cute quá|     1|   [a_bo, cute, quá]|
| A Bo có hát nữa à B|     1|[a_bo, có, hát, n...|
|A Boy chẳng bao h...|     1|[a_boy, chẳng, ba...|
|A Bình hát nhẹ nh...|     1|[a_bình, hát, nhẹ...|
|A Bộ có gia đình ...|     1|[a_bộ, có, gia_đì...|
|A Cao Thắng vs Ch...|     1|[a_cao_thắng, vs,...|
|A Chi Dân hát hay...|     1|[a, chi, dân, hát...|
|A Chi Dân vừa xa ...|     1|[a, chi_dân, vừa,...|
|     A Chị Simmy kìa|     1|  [a_chị, simmy_kìa]|
|A Chỗ này cũng ở ...|     1|[a_chỗ, này, cũng...|
|A D FRIEND NHE LUV U|     1|[a, d, friend, nh...|
|        A DI DA PHAT|     1|    [a_di, da, phat]|
|        A DI DA PHAT|     1|    [a_di, da, phat]|
|        A DI DA Phat|     1|  

In [208]:
from pyspark.ml.feature import (
    StopWordsRemover,
    Tokenizer,
    HashingTF,
    IDF
)
from pyspark.sql.functions import col
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipelinel
stopwords_file = open("./vietnamese-stopwords.txt", "r")
stopwords_list = stopwords_file.readlines()

# tokenizer = Tokenizer(inputCol="content", outputCol="content1")
# trainingData = trainingData.withColumn("content1", annotator.tokenize(col("content")))

stopwords_remover = StopWordsRemover(
    inputCol="content1",
    outputCol="content2",
    stopWords=stopwords_list,
)
hashing_tf = HashingTF(
    inputCol="content2",
    outputCol="term_frequency",
)
idf = IDF(
    inputCol="term_frequency",
    outputCol="features",
    minDocFreq=5
)
lr = LogisticRegression(labelCol="status")

sematic_analysis_pipeline = Pipeline(
    # stages=[tokenizer,stopwords_remover, hashing_tf, idf, lr]
    stages=[stopwords_remover, hashing_tf, idf, lr]
)

model = sematic_analysis_pipeline.fit(trainingData)

21/09/05 11:29:18 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:33:43 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:18 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:21 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:23 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:25 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:28 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:31 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:35 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:38 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:41 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/09/05 11:38:44 WARN DAGScheduler: Broadcasting larg

In [209]:
trained_df = model.transform(trainingData)

validationData = tokenize(validationData)
testData = tokenize(testData)

val_df = model.transform(validationData)
test_df = model.transform(testData)

trained_df.show()
val_df.show()
test_df.show()

21/09/05 11:42:37 WARN DAGScheduler: Broadcasting large task binary with size 5.0 MiB
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|             content|status|            content1|            content2|      term_frequency|            features|       rawPrediction|         probability|prediction|
+--------------------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                   A|     2|                 [a]|                 [a]|(262144,[107107],...|(262144,[107107],...|[-6.2442014526571...|[1.58497780600552...|       2.0|
|                   A|     2|                 [a]|                 [a]|(262144,[107107],...|(262144,[107107],...|[-6.2442014526571...|[1.58497780600552...|       2.0|
|                   A|     2|                 [a]|                 [a]|(262144,

In [210]:
from pyspark.ml.evaluation import (
    RegressionEvaluator,
    MulticlassClassificationEvaluator,
)

evaluator = MulticlassClassificationEvaluator(labelCol="status", metricName="accuracy")
accuracy_val = evaluator.evaluate(val_df)
accuracy_test = evaluator.evaluate(test_df)

print(f"Validation accuracy: {accuracy_val*100:.5f}%")
print(f"Test accuracy: {accuracy_test*100:.5f}%")

21/09/05 11:43:40 WARN DAGScheduler: Broadcasting large task binary with size 5.0 MiB
21/09/05 11:44:27 WARN DAGScheduler: Broadcasting large task binary with size 5.0 MiB
Validation accuracy: 88.42479%
Test accuracy: 88.51627%


In [211]:
model.write().overwrite().save('./model/')

21/09/05 11:45:19 WARN TaskSetManager: Stage 455 contains a task of very large size (4184 KiB). The maximum recommended task size is 1000 KiB.
